In [ ]:
%%capture
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

#Import

In [ ]:
from google.colab.drive import mount
from gcpds.databases import GIGA_MI_ME
import numpy as np
from MI_EEG_ClassMeth.Preprocessing import butterworth_digital_filter, ica_artiact_removal, spherical_spline_surface_laplacian
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
from mne.channels import make_standard_montage
from mne import create_info
import os
from time import time
from pickle import dump

#Mount Drive

In [ ]:
mount("/content/drive", force_remount=True)

Mounted at /content/drive


#Get Data

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/labels/')

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/ica/')

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/'
seed = 23
db = GIGA_MI_ME('/content/drive/Shareddrives/GCPDS/databases/GIGA-MI_ME/')
subjects = np.arange(db.metadata['subjects']) + 1
subjects = np.delete(subjects,[28,33])
fs = db.metadata['sampling_rate']


eeg_ch_names = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

index_eeg_chs = db.format_channels_selectors(channels=eeg_ch_names) - 1

montage = make_standard_montage(db.metadata['montage'])
info = create_info(eeg_ch_names, fs, len(eeg_ch_names)*['eeg'])
info.set_montage(montage)
t_mi = np.cast['uint16'](np.array([2, 5])*fs)

f_bank = np.asarray([[8,12],[12,15],[15,20],[18,40]])
vwt = np.asarray([[0.5, 2.5]])
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# ------------------------------------------------------------------------------
t_i = time()

#for sbj in subjects:
for sbj in np.array([1]):
  db.load_subject(sbj)
  X, y = db.get_data(classes=['left hand mi', 'right hand mi']) 
  X = X[:,index_eeg_chs,:] #measurements units uV

  X = butterworth_digital_filter(X, N=5, Wn=1, btype='highpass', fs=fs)
  X = X[:,:,t_mi[0]:t_mi[1]]
  A = X[:,:3,:]

  y_dir = os.path.join(data_dir , 'labels/')
  np.save(y_dir + 'sbj' + str(sbj), y)
  
  for ica in ['none_', 'ica_']:
    if ica == 'ica_':
      X_clean, idx_noise_trials, W_unmixing, rho = ica_artiact_removal(X, A, seed=seed)
      with open(results_dir + 'ica/' + 'sbj' + str(sbj) + '.txt', 'wb') as f:
        dump({'idx_noise_trials':idx_noise_trials,
              'W_unmixing':W_unmixing,
              'rho':rho}, f)
    else:
      X_clean = X

    for sl in ['none', 'sl']:       
      if sl == 'sl':
        X_sl = spherical_spline_surface_laplacian(X_clean, info)
      else:
        X_sl = X_clean

      X_wt = tf_repr.transform(X_sl)  

      X_dir = os.path.join(data_dir , ica+sl + '/')
      np.save(X_dir + 'sbj' + str(sbj), X_wt)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 7.74 m


Note: Using the frontal channels like artifact templates generates that many trials are detected like noise.